In [1]:
# Import libraries

import pandas as pd
import numpy as np
import os
import requests
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from tqdm import tqdm

folderpath = r"../Data"

filepaths = [
    os.path.join(folderpath, name)
    for name in os.listdir(folderpath)
    if name.endswith(".csv")
]

sample_list = []

print(f"Found {len(filepaths)} CSV files.\n")

for f in tqdm(filepaths, desc="Sampling files"):
    df_temp = pd.read_csv(f, low_memory=False)

    sample_size = min(100000, len(df_temp))
    sample = df_temp.sample(n=sample_size, random_state=42)
    sample_list.append(sample)

df_citi = pd.concat(sample_list, ignore_index=True)

print("Final sample shape:", df_citi.shape)
df_citi.head()

Found 38 CSV files.



Sampling files: 100%|██████████| 38/38 [03:08<00:00,  4.96s/it]


Final sample shape: (3624920, 15)


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,date,avgTemp
0,C31A06A81423A14C,classic_bike,2022-01-18 07:40:25.346,2022-01-18 07:43:35.441,W 26 St & 8 Ave,6297.02,10 Ave & W 28 St,6459.04,40.747348,-73.997236,40.750664,-74.001768,member,NaN,NaN
1,6104B30E6CC08F21,classic_bike,2022-01-18 20:13:23.444,2022-01-18 20:57:37.652,Jay St & York St,4895.09,Clark St & Henry St,4789.03,40.701403,-73.986727,40.697601,-73.993446,member,NaN,NaN
2,127F7DF7F787F370,classic_bike,2022-01-01 02:26:22.776,2022-01-01 02:46:12.503,Greenwich St & Perry St,5922.04,E 13 St & Avenue A,5779.09,40.734980,-74.006970,40.729667,-73.980680,casual,NaN,NaN
3,FF12FDCC260D47C3,classic_bike,2022-01-06 18:30:28.542,2022-01-06 18:44:35.763,Lexington Ave & E 36 St,6313.10,W 26 St & 10 Ave,6382.05,40.747574,-73.978801,40.749718,-74.002950,member,NaN,NaN
4,D8402474AF50D5F1,classic_bike,2022-01-27 18:29:15.544,2022-01-27 18:30:52.077,31 St & 21 Ave,7202.07,31 St & Ditmars Blvd,7144.02,40.778130,-73.908420,40.776168,-73.910485,member,NaN,NaN


In [ ]:
# Instead of loading the full CitiBike 2022 dataset (which is too large),
# I loop over all monthly CSV files and randomly sample up to 100,000 rows from each.
# This keeps the data representative and avoids memory errors.

In [3]:
import os
Token = os.getenv("NOAA_TOKEN")


In [4]:
# Get the API for La Guardia Airport Jan 1st - Dec 31st 2022

r = requests.get(
    'https://www.ncdc.noaa.gov/cdo-web/api/v2/data?datasetid=GHCND&datatypeid=TAVG&limit=1000&stationid=GHCND:USW00014732&startdate=2022-01-01&enddate=2022-12-31',
    headers={'token': Token}
)

In [5]:
# Load the api response as a json
d = r.json()

In [7]:
d['results'][:3]

[{'date': '2022-01-01T00:00:00',
  'datatype': 'TAVG',
  'station': 'GHCND:USW00014732',
  'attributes': 'H,,S,',
  'value': 116},
 {'date': '2022-01-02T00:00:00',
  'datatype': 'TAVG',
  'station': 'GHCND:USW00014732',
  'attributes': 'H,,S,',
  'value': 114},
 {'date': '2022-01-03T00:00:00',
  'datatype': 'TAVG',
  'station': 'GHCND:USW00014732',
  'attributes': 'H,,S,',
  'value': 14}]

In [8]:
# Secure all items in the response that correspond to TAVG

avg_temps = [item for item in d['results'] if item['datatype']=='TAVG']

In [36]:
# Get only the date field from all average temperature readings

data_temp = [item['date'] for item in avg_temps]

In [37]:
# Get the temperature from all average temperature readings

temps = [item['value'] for item in avg_temps]

In [38]:
# Put the results in a dataframe

df_temps = pd.DataFrame()

In [39]:
# Convert dates and temperatures
from datetime import datetime

df_temps['date'] = [datetime.strptime(d, "%Y-%m-%dT%H:%M:%S") for d in data_temp]
df_temps['avgTemp'] = [float(v)/10.0 for v in temps]

In [40]:
df_temp.tail()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
592044,1F223EDAFF420AE3,electric_bike,2022-12-01 20:26:45.847,2022-12-01 20:30:46.012,Avenue D & E 3 St,5436.09,Stanton St & Chrystie St,5523.02,40.720701,-73.977939,40.722293,-73.991475,member
592045,CFA5C560ACB73B8E,classic_bike,2022-12-26 13:46:34.237,2022-12-26 13:52:43.900,43 Ave & 47 St,6209.05,39 Ave & 45 St,6401.03,40.744806,-73.917290,40.749478,-73.918265,member
592046,11C8C5E0DB947B07,classic_bike,2022-12-01 05:56:14.903,2022-12-01 06:06:10.357,Avenue D & E 3 St,5436.09,Bleecker St & Crosby St,5679.08,40.720828,-73.977932,40.726156,-73.995102,member
592047,5B9B083C534A5964,classic_bike,2022-12-02 11:54:15.871,2022-12-02 12:01:00.747,Montague St & Clinton St,4677.06,Sands St & Jay St,4821.03,40.694271,-73.992327,40.700119,-73.986200,member
592048,91C286C462F89A50,classic_bike,2022-12-18 13:35:22.574,2022-12-18 13:37:27.193,Montague St & Clinton St,4677.06,Cadman Plaza E & Tillary St,4677.01,40.694271,-73.992327,40.695977,-73.990149,member


In [41]:
df_temp.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,1361583B8DDD2C67,electric_bike,2022-12-07 17:46:15.155,2022-12-07 18:04:36.889,Vesey St & Church St,5216.06,E 11 St & 1 Ave,5746.14,40.712220,-74.010472,40.729538,-73.984267,member
1,3A7AC1ACDD926798,classic_bike,2022-12-18 13:11:31.463,2022-12-18 13:14:24.975,E 5 St & Ave A,5626.06,Ave A & E 14 St,5779.11,40.724790,-73.984301,40.730311,-73.980472,member
2,DAB816EF7AC4BDC7,electric_bike,2022-12-08 18:39:03.508,2022-12-08 18:45:39.109,6 Ave & W 33 St,6364.07,9 Ave & W 39 St,6644.08,40.748776,-73.988742,40.756404,-73.994101,member
3,E84E793996D396B3,electric_bike,2022-12-20 16:05:06.384,2022-12-20 16:07:32.967,E 5 St & Ave A,5626.06,Ave A & E 14 St,5779.11,40.725000,-73.984241,40.730311,-73.980472,member
4,DB6ECA2FFA40CCA2,classic_bike,2022-12-20 09:27:12.027,2022-12-20 09:30:37.044,E 5 St & Ave A,5626.06,Ave A & E 14 St,5779.11,40.724790,-73.984301,40.730311,-73.980472,member


In [ ]:
# Export data from New York La Guardia's station to CSV file

output_path = "../Data/laguardia_weather_2022.csv"

df_temps.to_csv(output_path, index=False)

print("Saved CSV to:", output_path)
print(df_temps.head())

Saved CSV to: ../Data/laguardia_weather_2022.csv
        date  avgTemp
0 2022-01-01     11.6
1 2022-01-02     11.4
2 2022-01-03      1.4
3 2022-01-04     -2.7
4 2022-01-05      3.2


In [43]:
print(df_citi.columns)
print(df_temps.columns)

Index(['ride_id', 'rideable_type', 'started_at', 'ended_at',
       'start_station_name', 'start_station_id', 'end_station_name',
       'end_station_id', 'start_lat', 'start_lng', 'end_lat', 'end_lng',
       'member_casual'],
      dtype='object')
Index(['date', 'avgTemp'], dtype='object')


In [44]:
df_citi['started_at'].dtype

dtype('O')

In [45]:
df_temps['date'].dtype

dtype('<M8[ns]')

In [46]:
df_citi['started_at'].head()

0    2022-01-18 07:40:25.346
1    2022-01-18 20:13:23.444
2    2022-01-01 02:26:22.776
3    2022-01-06 18:30:28.542
4    2022-01-27 18:29:15.544
Name: started_at, dtype: object

In [47]:
df_temps['date'].head()

0   2022-01-01
1   2022-01-02
2   2022-01-03
3   2022-01-04
4   2022-01-05
Name: date, dtype: datetime64[ns]

In [51]:
# CitiBike: string -> datetime 
df_citi['started_at'] = pd.to_datetime(df_citi['started_at'])
df_citi['date'] = df_citi['started_at'].dt.date   

# Weather: 
df_temps['date'] = pd.to_datetime(df_temps['date']).dt.date

In [52]:
print(df_citi['date'].dtype)
print(df_temps['date'].dtype)

object
object


In [53]:
print(df_citi['date'].head())
print(df_temps['date'].head())

0    2022-01-18
1    2022-01-18
2    2022-01-01
3    2022-01-06
4    2022-01-27
Name: date, dtype: object
0    2022-01-01
1    2022-01-02
2    2022-01-03
3    2022-01-04
4    2022-01-05
Name: date, dtype: object


In [59]:
# Merge CitiBike and Weather Data
df_citi_weather = pd.merge(
    df_citi,
    df_temps[['date', 'avgTemp']],
    on='date',
    how='left',
    indicator=True 
)

In [60]:
# Qucik check
df_citi_weather[['started_at', 'date', 'avgTemp']].head()
print(df_citi_weather.shape)

(3524555, 16)


In [62]:
df_citi_weather.head()


,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual,date,avgTemp,_merge
0,C31A06A81423A14C,classic_bike,2022-01-18 07:40:25.346,2022-01-18 07:43:35.441,W 26 St & 8 Ave,6297.02,10 Ave & W 28 St,6459.04,40.747348,-73.997236,40.750664,-74.001768,member,2022-01-18,1.8,both
1,6104B30E6CC08F21,classic_bike,2022-01-18 20:13:23.444,2022-01-18 20:57:37.652,Jay St & York St,4895.09,Clark St & Henry St,4789.03,40.701403,-73.986727,40.697601,-73.993446,member,2022-01-18,1.8,both
2,127F7DF7F787F370,classic_bike,2022-01-01 02:26:22.776,2022-01-01 02:46:12.503,Greenwich St & Perry St,5922.04,E 13 St & Avenue A,5779.09,40.734980,-74.006970,40.729667,-73.980680,casual,2022-01-01,11.6,both
3,FF12FDCC260D47C3,classic_bike,2022-01-06 18:30:28.542,2022-01-06 18:44:35.763,Lexington Ave & E 36 St,6313.10,W 26 St & 10 Ave,6382.05,40.747574,-73.978801,40.749718,-74.002950,member,2022-01-06,4.9,both
4,D8402474AF50D5F1,classic_bike,2022-01-27 18:29:15.544,2022-01-27 18:30:52.077,31 St & 21 Ave,7202.07,31 St & Ditmars Blvd,7144.02,40.778130,-73.908420,40.776168,-73.910485,member,2022-01-27,-4.2,both


In [63]:
df_citi_weather['_merge'].value_counts(dropna=False)

_merge
both          3524490
left_only          65
right_only          0
Name: count, dtype: int64

In [69]:
df_citi_weather.to_csv("../Data/citibike_2022_with_weather_sample.csv", index=False)
print("Saved merged file.")

Saved merged file.
